In [1]:
%pip install torch
%pip install pytorch-lightning

You should consider upgrading via the '/Users/fliprise/my_notebooks/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import math
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

In [ ]:
# Base Data Module for PyTorch Lightning

class BaseDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32, split=0.8, *args, **kwargs):
        super().__init__()
        
        # Get the dataset using the get_dataset method (to be implemented in subclasses)
        self.ds_x, self.ds_y = self.get_dataset(*args, **kwargs)
        
        # Create a random permutation of indices to shuffle the dataset
        shuffler = np.random.permutation(self.ds_x.shape[0])
        
        # Shuffle both input features (ds_x) and labels (ds_y) using the same permutation
        self.ds_x = self.ds_x[shuffler]
        self.ds_y = self.ds_y[shuffler]
        
        # Set the batch size for data loading
        self.batch_size = batch_size
        
        # Calculate the split index for train/validation sets
        # split is a float between 0 and 1, representing the proportion of data for training
        self.split = int(self.ds_x.shape[0] * split)

    def train_dataloader(self):
        ds_X_train, ds_Y_train = self.ds_x[0:self.split], self.ds_y[0:self.split]
        return torch.utils.data.DataLoader(list(zip(ds_X_train, ds_Y_train)), batch_size=self.batch_size)

